In [13]:
import numpy as np
from sys import float_info
import scipy.special as sp
from mpmath import mpf, mp
from scipy.optimize import fmin
import math
import os as os
import csv
import sys
import pandas as pd
import scipy.optimize as op
from pandas.api.types import is_numeric_dtype
from numpy import isnan



mp.dps = 40 #this needs to be increased from the default value in order to get high enough numerical precision when calculating the lerchphi-function




def discrete_pdf_powerlaw(alpha, kmin, maxdeg):

    """ discrete power law pdf """
    
    degs = np.arange(kmin,maxdeg+1)
    p_degs = []
    
    C = 1 / sp.zeta(alpha, kmin)

    for deg in degs:
        p_deg = C * float(deg)**(-alpha)
        p_degs.append(p_deg)

    p_degs = np.asarray(p_degs)
    
    return degs, p_degs



def expected_R_div_n(p_reals, log_p_degs1, log_p_degs2):

    """ calculates the value of log-likelihood-ratio /n as n goes to infinity
        
        returns R/n, sigma"""
    
    mu1 = np.sum(p_reals*log_p_degs1)
    mu2 = np.sum(p_reals*log_p_degs2)
    diff = mu1-mu2
    
    R = np.sum(p_reals*(log_p_degs1-log_p_degs2))
    var = np.sum(p_reals*(log_p_degs1-log_p_degs2 - diff)**2)
    
    return R, math.sqrt(var)



def check_nan(cdf):
    if isnan(min(cdf)):
        print("'nan' in fit cumulative distribution values.", file=sys.stderr)
        print("Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.", file=sys.stderr)


In [14]:

def discrete_alpha(degs, p_degs, xmin):
    
    
    """ calculates the analytical limit value of the alpha estimate as n to infinity """

        
    def initial_parameters(degs):
        
        expected = np.sum(p_degs*(np.log(degs)-np.log(xmin)))
        alpha = 1 + 1/expected
        
        return alpha
    
    
    def in_range(alpha):
        
        inrange = True
        if not alpha > 1:
            inrange = False
            
        return inrange
            
    
    def fit_function(params):
        
        
        #here the ln(pdf) has just been calculated analytically a bit further
        
        alpha = params[0]
        expected_val = np.sum(p_degs*np.log(degs))
        L = -np.log(sp.zeta(alpha,xmin)) - alpha*expected_val    
        L = -L #negative for minimizing
        return L
    
    
    parameters, negative_loglikelihood, n_iter, funcalls, warnflag, = \
            fmin(
                lambda params: fit_function(params),
                initial_parameters(degs),
                full_output=1,
                disp=False)
   
    if warnflag == False:
        convstatus = True
    else:
        convstatus = False
    
    alpha = parameters[0]
    
    #####

    return alpha, convstatus



In [15]:
def opt_strexp(degs, p_degs):
    
    """ optimizes the parameters for SE for the theoretical distribution degs, p_degs """

    kmin = np.min(degs)
    ntail = len(degs)
    upper = 0.5
    lower = -0.5
    kmin = kmin + lower #update to accommodate Alstott discretization
    
    def in_range(lam,beta):
        
        inrange = True
        if not (lam > 0 and  0<beta): #<= 1):
            inrange = False
        return inrange

    def cdf_base(k,lam,beta):
        cdf = 1 - np.exp(-(lam*k)**beta)
        return cdf


    def CDF(k,lam,beta):
        num = cdf_base(k,lam,beta) - cdf_base(kmin,lam,beta)
        denom = 1 - cdf_base(kmin,lam,beta)
        cdf = num/denom
        check_nan(cdf)
        return cdf


    def PDF_alstott(k,lam,beta):

        return CDF(k+upper,lam,beta) - CDF(k+lower,lam,beta)


    def log_pdf(k,lam,beta):
        
        if not in_range(lam,beta): #checking if inside acceptable parameter range
            from numpy import tile
            return np.log(tile(10**float_info.min_10_exp, len(k)))

        val = PDF_alstott(k,lam,beta)
        val[val==0] = 10**float_info.min_10_exp

        return np.log(val)
        
    
    def initial_parameters():
        #with empirical data: (1/mean(data), 1)
        expected = np.sum(p_degs * degs) #expected deg
        return (1/expected,1)
  
    
    def fit_function(params):
        
        lam = params[0]
        beta = params[1]
        logl = log_pdf(degs,lam, beta)
        return -sum(p_degs*logl)
    
    
    parameters, negative_loglikelihood, n_iter, funcalls, warnflag, = \
            fmin(
                lambda params: fit_function(params),
                initial_parameters(),
                full_output=1,
                disp=False, maxiter = 1000)
    
    #print("n_iter", n_iter)
   
    if warnflag == False:
        convstatus = True
    else:
        convstatus = False
        print("strexp did not converge")
        
    lam,beta = parameters[0],parameters[1]
    
    LV = log_pdf(degs, lam, beta)
    
    return [parameters, LV, convstatus]
    



def opt_ln(degs, p_degs):

    
    kmin = np.min(degs)
    ntail = len(degs)
    upper = 0.5
    lower = -0.5
    kmin  = kmin + lower
    

    def in_range(sigma):
        
        inrange = True
        if not sigma > 0:
            inrange = False
        
        return inrange
    

    def cdf_base(k,mu,sigma):
        
        from numpy import sqrt, log
        from scipy.special import erf
        
        return  0.5 + ( 0.5 *
                erf((log(k)-mu) / (sqrt(2)*sigma)))


    def CDF(k,mu,sigma):
        num = cdf_base(k,mu,sigma) - cdf_base(kmin,mu,sigma)
        denom = 1- cdf_base(kmin,mu,sigma) #gives the ccdf
        cdf = num/denom
        check_nan(cdf)
        return cdf


    def PDF_alstott(k,mu,sigma):

        return CDF(k+upper,mu,sigma)-CDF(k+lower,mu,sigma)


    def log_pdf(k,mu,sigma):
        
        if not in_range(sigma):
            from numpy import tile
            return np.log(tile(10**float_info.min_10_exp, len(k)))

        val = PDF_alstott(k,mu,sigma)
        val[val==0] = 10**float_info.min_10_exp

        return np.log(val)
    
    
    def initial_parameters():
        
        #for simulated data, returns mean(logdata), std(logdata)
        logdata = np.log(degs)
        mu = np.sum(p_degs*logdata)
        var = np.sum(p_degs*(logdata-mu)**2)
        
        return mu, np.sqrt(var)

    
    def fit_function(params):

        logl = log_pdf(degs,params[0],params[1])
        return -sum(p_degs*logl) 
    
    
    parameters, negative_loglikelihood, n_iter, funcalls, warnflag, = \
            fmin(
                lambda params: fit_function(params),
                initial_parameters(),
                full_output=1,
                disp=False)
   
    if warnflag == False:
        convstatus = True
    else:
        convstatus = False
        print("ln did not converge")

    LV = log_pdf(degs,parameters[0],parameters[1])
    
    return [parameters, LV, convstatus]




def opt_exp(degs, p_degs):

    kmin = np.min(degs)
    
    def in_range(lam):
        
        inrange = True
        if not lam > 0:
            inrange = False
        return inrange
     
    def initial_parameters():
        #return 1/mean(data)
        expected = np.sum(p_degs*degs)
        return 1/expected
    
    def log_pdf(k,lam):
        
        if not in_range(lam):
            from numpy import tile
            return np.log(tile(10**float_info.min_10_exp, len(k)))
        
        C = (1 - np.exp(-lam)) * np.exp(lam * kmin)
        f = np.exp(-lam * k)
        val = f*C
        val[val==0] = 10**float_info.min_10_exp
        
        return np.log(val)

    
    def fit_function(params):
                             
        lam = params[0]
        logl = log_pdf(degs,lam)
        return -sum(p_degs*logl)
    
    
    parameters, negative_loglikelihood, n_iter, funcalls, warnflag, = \
            fmin(
                lambda params: fit_function(params),
                initial_parameters(),
                full_output=1,
                disp=False)
   
    if warnflag == False:
        convstatus = True
    else:
        convstatus = False
        print("exp did not converge")
    
    lam = parameters[0]  
    
    LV = log_pdf(degs,lam)
    
    return [lam, LV, convstatus]



def opt_trunc(degs, p_degs):
    

    kmin = np.min(degs)
    
    def in_range(alpha,lam):
        
        inrange = True
        if not lam > 0 or not alpha > 1:
            inrange = False
        return inrange
    

    def initial_parameters():
        
        
        expected_log_excess = np.sum(p_degs*(np.log(degs)-np.log(kmin)))
        alpha = 1 + 1/expected_log_excess

        expected = np.sum(p_degs*degs)
        lam = 1/expected
        
        return alpha, lam

    
    def pdf_discrete_normalizer(k,alpha,lam):
       
        from mpmath import lerchphi
        from mpmath import exp

        C = ( float(exp(kmin * lam) /
            lerchphi(exp(-lam), alpha, kmin)) )
        return C
    
    def pdf_base_function(k,alpha,lam):
        return k**(-alpha) * np.exp(-lam * k)
    
    
    def log_pdf(k,alpha,lam):
        
        if not in_range(alpha,lam):
            from numpy import tile
            return np.log(tile(10**float_info.min_10_exp, len(k)))
        
        f = pdf_base_function(k,alpha,lam)
        C = pdf_discrete_normalizer(k,alpha,lam)
        val = f*C
        val[val==0] = 10**float_info.min_10_exp
        
        return np.log(val)
    
    
    def fit_function(params):
                             
        logl = log_pdf(degs,params[0],params[1])
        return -sum(p_degs*logl)
    
    
    parameters, negative_loglikelihood, n_iter, funcalls, warnflag, = \
            fmin(
                lambda params: fit_function(params),
                initial_parameters(),
                full_output=1,
                disp=False)
   
    if warnflag == False:
        convstatus = True

    else:
        convstatus = False
        print("tr did not converge")
    
    
    alpha,lam = parameters[0], parameters[1] 
        
    LV = log_pdf(degs,alpha,lam)
    
    return [alpha,lam, LV, convstatus]

In [16]:
def analytical_clauset_results(degs, p_degs, kmin):

    """ Analytical results of the method of Clauset et al. as n goes to infinity
        degs and p_degs will be cut so that only degs >= kmin are included """

    
    kmin_ind = kmin-1

    #normalize p_degs so that vals >= xmin sum up to 1
    p_degs = p_degs[kmin_ind:] / np.sum(p_degs[kmin_ind:])
    degs = degs[kmin_ind:]
    
    #1. calculate alpha
        
    alpha,convstatus = discrete_alpha(degs,p_degs, kmin)

    if convstatus == False:
        print("WARNING, alpha did not converge")

    #2. calculate expected KS between the real dist and the fitted pl-model
    
    #get p_degs of pl
    p_degs_pl = get_discrete_pdf([alpha], kmin, maxdeg = degs[-1])
    
    cdfs_real = np.cumsum(p_degs)
    cdfs_pl = np.cumsum(p_degs_pl)
    KS  = np.max(abs(cdfs_real - cdfs_pl))
    
    R_list = []
    
    
    for altdist in ["exp", "strexp", "ln", "trunc"]:
    
        #a) calculate parameters of alternative distributions
        if altdist == "exp":
            [lam, LV, convstatus] = opt_exp(degs, p_degs)
            
        elif altdist == "strexp":
            [theta, LV, convstatus] = opt_strexp(degs,p_degs)
            
        elif altdist == "ln":
            [theta, LV, convstatus] = opt_ln(degs, p_degs)
            
        elif altdist == "trunc":
            [alpha_p, lam, LV, convstatus] = opt_trunc(degs, p_degs)
            
        else:
            print("Check disttype")
            
        if convstatus == False:
            print("DID NOT CONVERGE!!!", altdist)
            R_list.append(None)
            
        else:
            #calculate R/n
            
            LV[np.isnan(LV)] = 0
            R_div_n,sigma = expected_R_div_n(p_degs,np.log(p_degs_pl),LV)
            
            if altdist != "trunc":
                norm_R = 1./sigma * R_div_n
                R_list.append(norm_R)

            else:
                sample_R = R_div_n
                R_list.append(sample_R)
    
    
    return alpha, KS, R_list



In [ ]:
#Example on how to run the code


#get probability distribution of e.g. discrete power law (degs is of the form [1,2,3,4,5...,maxdeg])
degs,p_degs  = discrete_pdf_powerlaw(alpha=2.5,kmin=1,maxdeg=10**5)

kmin = 10 #you can choose an arbitrary kmin that will be used in the calculations (only values >= kmin will be considered)

#calling the function to calculate the results of the estimates as n goes to infinity
alpha, KS, [exp,se,ln,tr] = analytical_clauset_results(degs, p_degs, kmin) 

print(alpha,KS)